# Downloads for JunD binding prediction

If necessary, download prerequisites first.

In [ ]:
###################################################################################################If any one want to implement using google collab just run this cell before compiling other cells#################################################################################################! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/localimport syssys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
#!conda install --yes -c bioconda bedtools samtools
#!pip install janggu
#!conda install --yes tensorflow

In [ ]:
import os
from pybedtools import BedTool

In [ ]:
output = '../data'
os.makedirs(output, exist_ok=True)

## Downloading the data for JunD prediction


In [ ]:
!wget https://www.encodeproject.org/files/ENCFF446WOD/@@download/ENCFF446WOD.bed.gz -O {output}/jund_peaks.narrowPeak.gz
!wget https://www.encodeproject.org/files/ENCFF546PJU/@@download/ENCFF546PJU.bam -O  {output}/dnase_stam_encode.bam
!wget https://www.encodeproject.org/files/ENCFF059BEU/@@download/ENCFF059BEU.bam -O  {output}/dnase_stam_roadmap.bam

!wget http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/hg38-human/hg38.blacklist.bed.gz -O  {output}/hg38.blacklisted.bed.gz
!gunzip -f  {output}/hg38.blacklisted.bed.gz

# human genome sequence hg38
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz -O  {output}/hg38.fa.gz
!gunzip -f  {output}/hg38.fa.gz

!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes -O {output}/hg38.chrom.sizes

Bam files need to be indexed

In [ ]:
!samtools index {output}/dnase_stam_encode.bam
!samtools index {output}/dnase_stam_roadmap.bam

Prepare the JunD peaks

In [ ]:
BedTool(os.path.join(output, 'jund_peaks.narrowPeak.gz')).sort().merge().saveas(
    os.path.join(output, 'jund_raw_peaks.bed'))

Create region of interest (ROI)

In [ ]:
BedTool(os.path.join(output, 'jund_raw_peaks.bed')).slop(b=10000, 
                                                               g=os.path.join(output, 'hg38.chrom.sizes')) \
 .sort().merge().subtract(os.path.join(output, 'hg38.blacklisted.bed'))\
.saveas(os.path.join(output, 'roi_jund_extended.bed'))



In [ ]:
!janggu-trim {output}/roi_jund_extended.bed {output}/trim_roi_jund_extended.bed -divby 200